In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install chromadb huggingface_hub langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchain requests
!pip install langdetect
!pip install beautifulsoup4
!pip install langchain together


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 6.4 MB/s eta 0:00:

In [3]:
import os
# Set the USER_AGENT environment variable
os.environ['USER_AGENT'] = 'MyAppName/1.0'

In [4]:
# Import the necessary classes
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import requests
from langchain.llms import Together
import re
from bs4 import BeautifulSoup
from langchain.prompts import PromptTemplate

In [5]:
# Helper function to load or create a Chroma database
def create_or_load_chroma_db(persist_directory, embedding_model):
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory)
        print(f"Created directory for Chroma database at: {persist_directory}")

    try:
        # Load the existing ChromaDB
        chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
        print(f"Loaded existing Chroma database from {persist_directory}")
    except Exception as e:
        print(f"Error loading Chroma database: {e}")
        # If loading fails, create a new Chroma database
        print(f"Creating a new Chroma database at {persist_directory}")
        try:
            chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
            print("New Chroma database created successfully.")
        except Exception as e:
            print(f"Error creating Chroma database: {e}")
            raise  # Re-raise the exception after logging

    return chroma_db

In [6]:
def create_retriever(chroma_db, search_type="similarity", threshold=0.55, k=4, lambda_mult=0.25):
    retriever = chroma_db.as_retriever(
        search_type=search_type,
        relevance_score_threshold=threshold,
        k=k,
        lambda_mult=lambda_mult
    )
    return retriever

In [7]:
# Load the Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
chroma_db_path ="/content/drive/MyDrive/rag-model/second_embedding_chroma_db"
chroma_db_drive=create_or_load_chroma_db(chroma_db_path,embedding_model)

<ipython-input-7-0ed8d7d7bfbd>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-5-83189448e33f>:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)


Loaded existing Chroma database from /content/drive/MyDrive/rag-model/second_embedding_chroma_db


In [8]:
def clean_and_format_answers(answers):
    # Compile regex for cleaning HTML
    clean_html = re.compile('<.*?>')

    cleaned_text = ""
    for answer in answers:
        cleaned_text += f"Question: {answer['question']}\n\n"

        # Clean and format relevant documents
        for doc in answer['relevant_documents']:
            cleaned_doc = re.sub(clean_html, '', doc)  # Clean HTML tags
            cleaned_text += f"{cleaned_doc.strip()}\n\n"

    return cleaned_text.strip()  # Remove any extra spaces at the end


In [18]:
if __name__ == "__main__":
    # After ingestion function has been called
    retriever = create_retriever(chroma_db=chroma_db_drive)
    questions = [
 "What rights and protections are highlighted for vulnerable groups, such as women, children, and persons with disabilities, in the document?"]
    def get_answers(questions):
        answers = []
        for question in questions:
            results = retriever.get_relevant_documents(question)
            relevant_docs = [result.page_content for result in results]
            answers.append({
                "question": question,
                "relevant_documents": relevant_docs
            })
        return answers

In [19]:
answers = get_answers(questions)
cleaned_text = clean_and_format_answers(answers)

In [20]:
print(cleaned_text)

Question: What rights and protections are highlighted for vulnerable groups, such as women, children, and persons with disabilities, in the document?

(p) Concerned about the difficult conditions faced by persons with disabilities who are subject to multiple or aggravated forms of discrimination on the basis of race, colour, sex, language, religion, political or other opinion, national, ethnic, indigenous or social origin, property, birth, age or other status, 
     (q) Recognizing that women and girls with disabilities are often at greater risk, both within and outside the home of violence, injury or abuse, neglect or negligent treatment, maltreatment or exploitation,
     (r) Recognizing that children with disabilities should have full enjoyment of all human rights and fundamental freedoms on an equal basis with other children, and recalling obligations to that end undertaken by States Parties to the Convention on the Rights of the Child,

(n) Concerned about the difficult conditions

In [21]:
llm = Together(model="meta-llama/Llama-2-13b-chat-hf", together_api_key="481dc4b7f7e5a6bff243a6d6d6f7c896e83a909a5b52c5f3aaf663d291c8946a",temperature=0)


In [22]:
# template for human rights domain with comprehensive coverage
template = """
This is a conversation between a human and an AI assistant familiar with human rights.
{history}
Human: I will provide a text from the retrieved documents and the question asked. Please formulate a coherent response based on the information provided.
Be sure to highlight all important aspects of human rights mentioned in the text.
If specific articles or laws related to human rights are mentioned in the text, please refer to them explicitly.
In addition, be neutral in any response and make your primary reference the retrieved documents that I will send you.
If sources are available, please refer to the documents appropriately.
Text from the retrieved documents and the question asked:
{input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template,
)


In [23]:
# Define the history and input text
history = ""
# Generate the complete prompt
formatted_prompt = prompt.format(history=history, input=cleaned_text)
# Get the response from the model
response = llm(formatted_prompt)

# Print the response
print("AI Response:", response)

AI Response:  Based on the text provided, the following are the rights and protections highlighted for vulnerable groups, such as women, children, and persons with disabilities:

1. Protection from discrimination: The text emphasizes the need to protect persons with disabilities from multiple or aggravated forms of discrimination on the basis of race, color, sex, language, religion, political or other opinion, national, ethnic, indigenous or social origin, property, birth, age or other status.
2. Right to equal enjoyment of human rights: The text recognizes the right of children with disabilities to have full enjoyment of all human rights and fundamental freedoms on an equal basis with other children.
3. Protection from violence: The text highlights the fact that women and girls with disabilities are often at greater risk of violence, injury or abuse, neglect or negligent treatment, maltreatment or exploitation, both within and outside the home.
4. Gender perspective: The text emphasiz